In [1]:
import numpy as np
import pandas as pd
import os
import sys
import metrics as mt

In [2]:
split_char=","
sep_char="\t"
test_set_path = '../Dataset/test_set.csv'
sub_path = '../submissions/'
test_set = pd.read_csv(test_set_path, sep=sep_char,index_col=0)
test_set = test_set['items'].apply(lambda x: pd.Series([int(i) for i in str(x).split(split_char)] if x!='' else []))
#test_set = test_set.fillna(int(0))

In [ ]:
sub_names = [ 'sub_interactions.csv',
             'k_int_1_shrink_1_knn_750_KNN=50_pools=interactions.csv',
             'k_int_1_shrink_1_knn_750_KNN=100_pools=interactions.csv',
             'k_int_1_shrink_1_knn_750_KNN=500_pools=interactions.csv',
             'k_int_3_shrink_1_knn_750_KNN=50_pools=interactions.csv',
             'k_int_3_shrink_1_knn_750_KNN=100_pools=interactions.csv',
             'k_int_3_shrink_1_knn_750_KNN=500_pools=interactions.csv',
             'k_int_1_shrink_10_knn_750_KNN=50_pools=interactions.csv',
             'k_int_1_shrink_10_knn_750_KNN=100_pools=interactions.csv',
             'k_int_1_shrink_10_knn_750_KNN=500_pools=interactions.csv',
             'k_int_3_shrink_10_knn_750_KNN=50_pools=interactions.csv',
             'k_int_3_shrink_10_knn_750_KNN=100_pools=interactions.csv',
             'k_int_3_shrink_10_knn_750_KNN=500_pools=interactions.csv',
             'k_int_6_shrink_10_knn_750_KNN=50_pools=interactions.csv',
             'k_int_6_shrink_10_knn_750_KNN=100_pools=interactions.csv',
             'k_int_6_shrink_10_knn_750_KNN=500_pools=interactions.csv',
             'k_int_1_shrink_10_knn_500_KNN=50_pools=interactions.csv',
             'k_int_1_shrink_10_knn_500_KNN=100_pools=interactions.csv',
             'k_int_1_shrink_10_knn_500_KNN=500_pools=interactions.csv',
             'k_int_1_shrink_1_knn_500_KNN=50_pools=interactions.csv',
             'k_int_1_shrink_1_knn_500_KNN=100_pools=interactions.csv',
             'k_int_1_shrink_1_knn_500_KNN=500_pools=interactions.csv',
             
            ]
            
results = pd.DataFrame([['',0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]],\
                       columns=list(['name', 'recall', 'map2', 'map5', 'map10','map20','NDCG','user success','competition_score']))


In [37]:
sub_names = ['k_int_1_shrink_20_knn_750_prova_user_KNN=50_pools=interactions.csv',
             'k_int_1_shrink_20_knn_750_prova_user_KNN=100_pools=interactions.csv',
             'k_int_1_shrink_20_knn_750_prova_user_KNN=500_pools=interactions.csv']
            
for sub_name in sub_names:
    submission = pd.read_csv(os.path.join(sub_path, sub_name), sep=sep_char,index_col=0)
    submission = submission.fillna('')
    submission.columns = ['items']
    sub_ranks = submission['items'].apply(lambda x: pd.Series([int(i) for i in str(x).split(split_char)] if x!='' else []))
    print('EVALUATING SUBMISSION ' + sub_name)
    #print('Running evaluation over {} users'.format(len(test_set)))
    #print('Your submission contains predictions for {} of them'.format(submission.shape[0]))
    metrics = mt.compute_metrics(sub_ranks,test_set,k=30)
    cscore = mt.challenge_score(sub_ranks,test_set,k=30)
    print('Average Recall: '), metrics[0]
    #print('Mean average precision at 2: '), metrics[2]
    #print('Mean average precision at 5: '), metrics[3]
    #print('Mean average precision at 10: '), metrics[4]
    print('Mean average precision at 20: '), metrics[5]
    #print('Average user success: '), metrics[1]
    print('Normalized Discount Cumulative Gain: '), metrics[6]
    print('Competition score: '), cscore
    print(' ')
    sub_results = pd.DataFrame([[sub_name,metrics[0],metrics[2],metrics[3],metrics[4],metrics[5],metrics[6],metrics[1],cscore]],\
                               columns=['name', 'recall', 'map2', 'map5', 'map10','map20','NDCG','user success','competition_score'])
    results = results.append(sub_results)
    

EVALUATING SUBMISSION k_int_1_shrink_20_knn_750_prova_user_KNN=50_pools=interactions.csv
Average Recall:  0.111176868731
Mean average precision at 20:  0.0496052008045
Normalized Discount Cumulative Gain:  0.0780362170097
Competition score:  94669.7070796
 
EVALUATING SUBMISSION k_int_1_shrink_20_knn_750_prova_user_KNN=100_pools=interactions.csv
Average Recall:  0.11584713181
Mean average precision at 20:  0.0495372328948
Normalized Discount Cumulative Gain:  0.0790928741401
Competition score:  96612.5930287
 
EVALUATING SUBMISSION k_int_1_shrink_20_knn_750_prova_user_KNN=500_pools=interactions.csv
Average Recall:  0.116505167018
Mean average precision at 20:  0.0488307887079
Normalized Discount Cumulative Gain:  0.0785464717175
Competition score:  96460.3667369
 


In [33]:
results.to_csv("results_comparison.csv")

In [35]:
results.sort_values('recall')

,name,recall,map2,map5,map10,map20,NDCG,user success,competition_score
0,,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
0,k_int_6_shrink_10_knn_750_KNN=50_pools=interac...,0.082216,0.035450,0.031665,0.031595,0.032253,0.053322,0.183000,68321.283741
0,k_int_1_shrink_1_knn_750_KNN=50_pools=interact...,0.082790,0.040025,0.036476,0.036122,0.036562,0.057181,0.186100,70399.743986
0,k_int_1_shrink_1_knn_500_KNN=50_pools=interact...,0.082790,0.040025,0.036476,0.036122,0.036562,0.057181,0.186100,70399.743986
0,k_int_1_shrink_10_knn_500_KNN=50_pools=interac...,0.082796,0.039925,0.036364,0.036074,0.036510,0.057148,0.186200,70381.855097
0,k_int_1_shrink_10_knn_750_KNN=50_pools=interac...,0.082796,0.039925,0.036364,0.036074,0.036510,0.057148,0.186200,70381.855097
0,k_int_1_shrink_20_knn_750_KNN=50_pools=interac...,0.083086,0.039950,0.036373,0.036065,0.036515,0.057226,0.186700,70546.962240
0,k_int_3_shrink_1_knn_750_KNN=50_pools=interact...,0.083411,0.037700,0.034101,0.034023,0.034603,0.055675,0.185800,69849.421804
0,k_int_6_shrink_10_knn_750_KNN=500_pools=intera...,0.083442,0.034750,0.031065,0.030904,0.031713,0.052974,0.184800,68414.293336
0,k_int_3_shrink_10_knn_750_KNN=50_pools=interac...,0.083535,0.037625,0.034066,0.034010,0.034587,0.055686,0.185800,69857.517042


In [9]:
print('Running evaluation over {} users'.format(len(test_set)))
print('Your submission contains predictions for {} of them'.format(submission.shape[0]))
metrics = mt.compute_metrics(sub_ranks,test_set,k=5)
print('Average Recall: '), metrics[0]
print('Mean average precision at 2: '), metrics[2]
print('Mean average precision at 5: '), metrics[3]
print('Mean average precision at 10: '), metrics[4]
print('Mean average precision at 20: '), metrics[5]
print('Average user success: '), metrics[1]
print('Normalized Discount Cumulative Gain: '), metrics[6]
print('Competition score: '), mt.challenge_score(sub_ranks,test_set,k=5)

Running evaluation over 10000 users
Your submission contains predictions for 10000 of them
Average Recall:  0.0311974332891
Mean average precision at 2:  0.0062
Mean average precision at 5:  0.00578572222222
Mean average precision at 10:  0.00631907440476
Mean average precision at 20:  0.00651285835925
Average user success:  0.0606
Normalized Discount Cumulative Gain:  0.014136201952
Competition score:  21921.4866578


In [5]:
print('Competition score: '), mt.challenge_score(sub_ranks,test_set,k=5)

Competition score:  21921.4866578


In [6]:
submission = pd.read_csv(test_set_path, sep=sep_char,index_col=0)
submission = submission.fillna('')
submission.columns = ['items']
sub_ranks = submission['items'].apply(lambda x: pd.Series([int(i) for i in str(x).split(split_char)] if x!='' else []))
#sub_ranks = sub_ranks.fillna(int(0))
print('PERFECT SCORE:')
metrics = mt.compute_metrics(sub_ranks,test_set,k=30)
print('Average Recall: '), metrics[0]
print('Mean average precision at 2: '), metrics[2]
print('Mean average precision at 5: '), metrics[3]
print('Mean average precision at 10: '), metrics[4]
print('Mean average precision at 20: '), metrics[5]
print('Average user success: '), metrics[1]
print('Normalized Discount Cumulative Gain: '), metrics[6]
print('Competition score: '), mt.challenge_score(sub_ranks,test_set,k=5)

PERFECT SCORE:
Average Recall:  1.0
Mean average precision at 2:  1.0
Mean average precision at 5:  1.0
Mean average precision at 10:  1.0
Mean average precision at 20:  1.0
Average user success:  1.0
Normalized Discount Cumulative Gain:  1.0
Competition score:  740106.0


In [6]:
submission

,items
user_id,
2588672,"1070094,1686228,490987,767198,2683814"
884740,"687732,1729618,2593483,933025,1707604"
2424838,"1244471,448383,2197180,1231437,1096754"
2097161,"709066,1575953,2399675,2433034,1445481"
262160,"1672383,300053,1244196,2308232,1503518"
2752531,"2682865,2016338,311542,2377969,1192552"
2752532,"388112,1768483,405633,1386412,2413494"
1966101,
1343511,"2129475,1058763,898128,742821,1729618"


Prove
--

In [36]:
import math
def dcg(recommended_items,relevant_items):
    is_relevant = np.in1d(recommended_items[:30], relevant_items, assume_unique=True).astype(np.float32)[:30]
    pos = 1
    gain = 0
    for i in is_relevant:
        gain+= (math.pow(2,i)-1)/math.log(pos+1)
        pos+=1
    return gain

def ndcg(recommended_items,relevant_items):
    return dcg(recommended_items,relevant_items)/float(dcg(relevant_items,relevant_items))

uid = 302
recommended_items = sub_ranks.ix[uid].values
relevant_items = test_set.ix[uid].values   

print dcg(recommended_items,relevant_items)
print ndcg(recommended_items,relevant_items)

6.0
4.76762569144
1.0


In [23]:
math.log(1,2)

0.0

In [7]:
#Metrics
import numpy as np
import pandas as pd

def ap_at_k(recommended_items, relevant_items, k):
    is_relevant = np.in1d(recommended_items[:k], relevant_items, assume_unique=True).astype(np.float32)
    total_relevant = len(relevant_items[~np.isnan(relevant_items)])
    score = np.sum(np.cumsum(is_relevant) * is_relevant / np.arange(1, k+1)) / min(k, total_relevant)
    assert 0 <= score <= 1
    return score

def precision_at_k(recommended_items, relevant_items, k):
	return float(np.intersect1d(recommended_items[:k], relevant_items).shape[0]) / k

def recall(recommended_items, relevant_items):
    if relevant_items.shape[0] == 0: 
        return 0.0
    total_relevant = len(relevant_items[~np.isnan(relevant_items)])
    return float(np.intersect1d(recommended_items[:30], relevant_items).shape[0]) / min(30, total_relevant)

def user_success(recommended_items, relevant_items):
	return float(np.intersect1d(recommended_items[:30], relevant_items).shape[0] > 0)

def compute_metrics(S, T, k=30):
    recall_avg, user_success_avg, map_at_2, map_at_5, map_at_10, map_at_20 = 0.0, 0.0, 0.0, 0.0, 0.0, 0.0
    for uid in S.index:
        #parse ids
        recommended_items = S.ix[uid].values
        relevant_items = T.ix[uid].values    
        
        #compute metrics for a user
        prec = precision_at_k(recommended_items, relevant_items, 5)
        rec = recall(recommended_items, relevant_items)      
        usucc = user_success(recommended_items, relevant_items)
        
        #update averages
        recall_avg += rec
        user_success_avg += usucc
        
        map_at_2 += ap_at_k(recommended_items, relevant_items, 2)
        map_at_5 += ap_at_k(recommended_items, relevant_items, 5)
        map_at_10 += ap_at_k(recommended_items, relevant_items, 10)
        map_at_20 += ap_at_k(recommended_items, relevant_items, 20)
        
        
    recall_avg /= S.shape[0]
    user_success_avg /= S.shape[0]
    map_at_2 /= S.shape[0]
    map_at_5 /= S.shape[0]
    map_at_10 /= S.shape[0]
    map_at_20 /= S.shape[0]
    
    return (recall_avg, user_success_avg, map_at_2, map_at_5, map_at_10, map_at_20)

In [8]:
sub_ranks

,0,1,2,3,4,5,6,7,8,9,...,206,207,208,209,210,211,212,213,214,215
user_id,,,,,,,,,,,,,,,,,,,,,
242,794156,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
302,2816087,405029,386805,224811,1501340,281913,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
517,389202,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
693,2755404,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
927,621104,2386634,1688480,2200671,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1021,2086845,158270,861853,51035,1891281,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1286,1252522,26793,159660,1073391,167049,1325179,293384,1057330,1859201,866523,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1509,343132,1686571,388915,21586,828489,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1797,1652172,888658,1211336,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
df3 = pd.DataFrame([[0,0,1]],      columns=['a', 'b', 'c'])

In [22]:
df3

,a,b,c
0,0,0,1
